In [2]:
import moviepy.editor as me 
from moviepy.editor import *
import pydub
from pydub import AudioSegment
import os
import numpy as np

### Separating out audio files from video for later selective muting

In [2]:
stimuli_path = "/Users/f004p74/Documents/dartmouth/projects/fSEND/stimuli/narrated-thought/"
dirlist = os.listdir(stimuli_path)
vidlist = [x for x in dirlist if ('.mp4') in x]

In [3]:
for a,b in enumerate(vidlist):
    b = b.split(".")
    vidlist[a] = b[0]

In [8]:
for i in vidlist:
    video = me.VideoFileClip(stimuli_path+str(i)+".mp4")
    video.audio.write_audiofile(stimuli_path+str(i)+".wav")

MoviePy - Writing audio in /Users/f004p74/Documents/dartmouth/projects/fSEND/stimuli/narrated-thought/stutter.wav


MoviePy - Done.
MoviePy - Writing audio in /Users/f004p74/Documents/dartmouth/projects/fSEND/stimuli/narrated-thought/physical.wav


MoviePy - Done.


### Removing slices of audio from the Narrated Thought videos
After isolating the audio from the videos, I create a function below which allows me to type in the minute and second that a particular scene begins and ends that I want to manipulate using a voice removing model. 

In [25]:
stimuli_path = "/Users/f004p74/Documents/dartmouth/projects/fSEND/stimuli/narrated-thought/"
dirlist = os.listdir(stimuli_path)
audiolist = [x for x in dirlist if ('physical.mp3') in x]
audio = AudioSegment.from_mp3(stimuli_path+audiolist[0])

In [39]:
def clip_audio (start_min, start_sec, stop_min, stop_sec):
    start_minute_conv = start_min * 60000
    start_second_conv = start_sec * 1000
    stop_minute_conv = stop_min * 60000
    stop_second_conv = stop_sec * 1000
    
    start = start_minute_conv + start_second_conv
    stop = stop_minute_conv + stop_second_conv
    
    segment = audio[start:stop]
    segment.export(stimuli_path+"segments/physical-v2/physical_"+str(start)+"_"+str(stop)+".mp3", format="mp3")
    
    return("start:",start,"stop:",stop)

In [49]:
clip_audio(5,3,5,14)

('start:', 303000, 'stop:', 314000)

## Adjusting 'quiet' clips
Two clips in particular were almost too quiet to hear. Here, I import the videos, multiply the audio by 3, and then (outside of this notebook: vocalremover.org by Melnik Dmitry) clean the file by boosting the voice signal and reducing the background noise. 

In [5]:
from moviepy.audio.fx.volumex import volumex

In [30]:
def boost_volume (video_path, name):
    clip = me.VideoFileClip(video_path)
    newclip = volumex(clip, 3.0)
    newclip.write_videofile(str(name)+"_vid_3x_audio.mp4",
                            codec='libx264', 
                            audio_codec='aac', 
                            temp_audiofile='temp-audio.m4a', 
                            remove_temp=True)
    
def replace_audio (video_path, audio_path, name):
    clip = me.VideoFileClip(video_path)
    audio = AudioFileClip(audio_path)
    
    final_clip = clip.set_audio(audio)
    final_clip.write_videofile(str(name)+".mp4",
                        codec='libx264', 
                        audio_codec='aac', 
                        temp_audiofile='temp-audio.m4a', 
                        remove_temp=True)
    